In [40]:
''' logistic regression '''

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 기본적으로 regression + classification(sigmoid 함수를 적용, 0.5보다 크면 1, 0.5보다 작으면 0)

# cost 함수는 로컬 미니멈이 있는 MSE 대신 cross-entropy
# C(H(x), y) = -log(H(x)) if y = 1 or -log(1-H(x)) if y = 0 = -ylog(H(x)) - (1-y)log(1-H(x))
# cross entropy를 이용한 경사하강법 이용


x_data = [[1, 2],
          [2, 3],
          [3, 1],
          [4, 3],
          [5, 3],
          [6, 2]]
y_data = [[0],
          [0],
          [0],
          [1],
          [1],
          [1]] # 이진 분류 logistic regression이기 때문에 Y데이터는 0 또는 1로 이루어져있어야 함

X = tf.placeholder(tf.float32, shape=[None, 2])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([2, 1]), name='weight') # x 데이터가 n x 2고 하나의 값을 내야 하므로 w는 2 x 1
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.sigmoid(tf.matmul(X,W)+b) # H(x) = 1 / 1 + e^(-XW + b) , hypothsis = tf.div(1. , 1. + tf.exp(-tf.matmul(X,W)))

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1-Y)* tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype = tf.float32) # 1과 0으로 변환하는데 있어 기준점이 되는 값 0.5 설정
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32)) # tf.equal(A,B) 은 두개의 값이 같으면 True, 다르면 False 반납 -> 이를 reduced_mean으로 평균을 냄

#학습
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, cost_val)

    # 정확성
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)




0 1.4671665
200 0.4494747
400 0.43163657
600 0.41543177
800 0.40055165
1000 0.38676262
1200 0.3738906
1400 0.36180663
1600 0.3504158
1800 0.3396463
2000 0.3294426
2200 0.31975967
2400 0.31056002
2600 0.30181113
2800 0.29348418
3000 0.28555313
3200 0.27799413
3400 0.27078506
3600 0.2639055
3800 0.2573361
4000 0.251059
4200 0.2450576
4400 0.23931623
4600 0.23382014
4800 0.22855544
5000 0.22350943
5200 0.21867
5400 0.21402574
5600 0.20956612
5800 0.2052813
6000 0.20116186
6200 0.19719917
6400 0.19338499
6600 0.18971176
6800 0.18617219
7000 0.18275948
7200 0.17946748
7400 0.17629011
7600 0.17322175
7800 0.17025721
8000 0.16739155
8200 0.16461995
8400 0.1619382
8600 0.15934218
8800 0.15682787
9000 0.15439166
9200 0.1520301
9400 0.14973988
9600 0.14751787
9800 0.14536124
10000 0.14326717

Hypothesis:  [[0.02816371]
 [0.15519828]
 [0.29229802]
 [0.7871852 ]
 [0.9431742 ]
 [0.98138404]] 
Correct (Y):  [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]] 
Accuracy:  1.0


In [41]:
""" Softmax Regression : 0과 1 사이의 확률로 나타냄
텐서플로우에서는 hypothesis = tf.nn.softmax(tf.matmul(X,W) + b) ; 이때 softmax의 인수를 logit 이라고도 함

이때 loss function은 역시 cross-entropy: -(1/m)∑Y_i * log(W*X_i + b) 
"""


x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])

nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
            _, cost_val = sess.run([optimizer, cost], feed_dict={X: x_data, Y: y_data})

            if step % 200 == 0:
                print(step, cost_val)

    print('--------------')
    # 테스트 & 원핫코딩
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1))) # argmax는 softmax 중 몇번째에 있는 elemtent가 가장 높은 확률을 가지고 있는지 반환

    print('--------------')
    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))

    print('--------------')
    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))

    print('--------------')
    all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))

0 9.443914
200 0.63929695
400 0.51892287
600 0.41869053
800 0.32393542
1000 0.25158808
1200 0.22632937
1400 0.20577154
1600 0.18854411
1800 0.17389613
2000 0.1612928
--------------
[[8.2720080e-03 9.9172217e-01 5.7714356e-06]] [1]
--------------
[[0.83200955 0.16245934 0.00553115]] [0]
--------------
[[1.2663491e-08 3.2328328e-04 9.9967670e-01]] [2]
--------------
[[8.2720080e-03 9.9172217e-01 5.7714356e-06]
 [8.3200955e-01 1.6245934e-01 5.5311495e-03]
 [1.2663491e-08 3.2328328e-04 9.9967670e-01]] [1 0 2]
